# Visualizing Data in Amazon QuickSight

This lab focuses on putting our data to work, by visualizing it and drawing insights.

## Amazon QuickSight

Amazon QuickSight is a cloud powered dashboard that can be used to deliver insights into your data. Quicksight allows you share dashboards, email reports, and embedded analytics. This allows you to show your company and your customers in an elegant way. By taking your data and visually displaying the questions you want answered, you can gain relevant insights your company.ed Data

## Creating the Database

Navigate to [Amazon RDS](https://us-west-2.console.aws.amazon.com/rds/home?region=us-west-2#databases:) and find the created database. It can take up to 10 minutes to complete provisioning. Record the host (endpoint) to update in the code block below once the database is ready.

![Hostname](./assets/lab2/Lab2_Hostname.png)

### Populating the Database with Mock Data

Here's some code to populate the database

In [2]:
! pip install mysql-connector

import mysql.connector, csv
## Make sure the Host matches your RDS instance captured in the previous step
mydb = mysql.connector.connect(
  host="inventory.cxz4yqgmagjq.us-west-2.rds.amazonaws.com", 
  user="admin",
  passwd="demotest123"
)

cursor = mydb.cursor()
cursor.execute("CREATE DATABASE IF NOT EXISTS inventory_quicksight")
mydb.database = "inventory_quicksight"
cursor.execute("CREATE TABLE IF NOT EXISTS product_table_qs (ProductName varchar(50),Description varchar(255),ProductSku bigint(12),QuantityInStock int(5),ReorderLevel int(5),WholeSaleCost decimal(8,2),RetailCost decimal(8,2));")

print("Database setup complete.")
with open('./data_files/Lab2_Inventory.csv') as datafile:
    csv_data = csv.reader(datafile)
    next(csv_data) 
    for row in csv_data:
        cursor.execute ("INSERT INTO product_table_qs (ProductName,Description,ProductSku,QuantityInStock,ReorderLevel,WholeSaleCost,RetailCost) VALUES (%s, %s, %s, %s, %s, %s, %s)",row)
    mydb.commit()
print("Success! Data inserted")
mydb.close()

You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Database setup complete.
Success! Data inserted


Cool right? Now that we have our data let's use Amazon Quicksight to visualize it.

## Creating Visualizations in Quicksight

### Creating an Amazon QuickSight Standard Account

The QuickSight service is not enabled by default and needs to be enabled in the AWS account you are using the first time QuickSight is used. If your account already has QuickSight enabled, you can skip to the next section.

1. Navigate to [the QuickSight start page](https://us-west-2.quicksight.aws.amazon.com/sn/start).

1. Click **Sign up for QuickSight**:
    
    ![Sign up for QuickSight](./assets/lab2/quicksight_signup.png)

1. Select the **Standard** Edition of QuickSight and click **Continue**.

    On the page that appears you will see the following error which can safely be ignored:

    ![Sign up error](./assets/lab2/signup_error.png)

    Despite the message, you can subscribe to QuickSight with all that is required for this lab.

1. In the **Create your QuickSight account** form, set the following values before clicking **Finish**:

    - **QuickSight account name**: Enter _calabs-#####_ where _#####_ is a unique 5-digit number
    - **Notification email address**: Enter <em>fake@quicksight.email</em>.  Email notifications are not important for this lab.
    - Uncheck all of the checkboxes. You will manually configure the connection to your RDS database in the next section.

    ![QuickSight Account Configuration](./assets/lab2/quicksight_config.png)

    After a minute you will see the following confirming your QuickSight Standard account is ready to use:

    ![QuickSight account ready](./assets/lab2/account_ready.png)

1. Click **Go to Amazon QuickSight**.

### Connect To Our Database

First navigate to the [Amazon QuickSight start page](https://us-west-2.quicksight.aws.amazon.com/sn/start).
In the top left, click the __New Analysis__ button. We'll be using the new dataset we created earlier, so next click on __New Dataset__.
There are a lot of options to choose from, from Twitter, to data files, to databases.
We'll be sticking with our MySQL database, so lets connect to that.
Click on the __MySQL__ button
You'll be preseted with a form to connect to the database.

![QuickSight Datastore Setup](./assets/lab2/Lab2_QuickSightSetup.png)
- **Datasource Name**: Enter _Inventory_. This is the name to be referenced when viewing the datasource on Quicksight.
- **Connection Type**: Leave this as _Public Network_ for now.
- **Database Server**: Enter the endpoint of the RDS instance you used for the Host variable in the code block above (don't include the quotation marks). This is the connection URL.
- **Port**: Enter MySQL's default port number _3306_
- **Database Name**:  Enter <em>inventory_quicksight</em> (which is the name of the database created by the code block above).
- **Username**: Enter _admin_.
- **Password**: Enter _demotest123_.

You can test the database connection before continuing by clicking the __Validate Connection__ button in the bottom left. Next click __Create Datasource__ to have QuickSight start using your database. You will be prompted to select the table that we should visualize. Click the radio button next to `product_table_qs` then press the __Select__ button.  Finally ensure `Import to SPICE for quicker analytics` is selected and press the __Visualize__ button.

## Creating Your First Analysis

![QuickSight Visualize Page](./assets/lab2/Lab2_QuickSight_Visualize.png)

You should now see the QuickSight Visualize page, if you do not go back to the [Amazon QuickSight start page](https://us-west-2.quicksight.aws.amazon.com/sn/start)  click on the button in the top left corner of your screen that says __New analysis__, select the datasource `product_table_qs` then click the __Create Analysis__ button.

Now that we are in the visualize page there are a lot of different options to work with.
Let's go over a few of the sections before diving in.

- **Fields**: These reflect the columns of the table in the database.
- **Visual Types**: This is how your data will be represented. This can be from a simple sum, to a chart/graph, or even a heat map.
- **Sheets**: These allow for multiple visuals to be stored together in a single page. To keep things simple, we'll be working with only one sheet.


#### **Lab:** Lets build our first QuickSight Visualization!

First in the _Visual Types_ click the __Doughnut Chart__.  Notice the __Field wells__ that appeared at the top. This is where you'll put the data being represented.

Drag `ProductName` from the fields into the __Group/Color__ box. Next, drag `QuantityInStock` to the __Value__ box.
The graph will be automatically generated based on the data in the database.  For the `QuantityInStock` __Value__ select the drop down arrow and change the aggregate to __Max__.
You should see a graph similar to the following on your screen.

![QuickSight Visualization](./assets/lab2/Lab2_QuickSight_Visualization_1.png)

Thats it! You've just created your first visualization. Try changing the __Visual Type__ of this data and see how it's represented. You might need to add addition fields to the __Field wells__ to make it populate correctly.

**Quick Note**: QuickSight, by default, has an automatic save feature enabled by default for each analysis. This can be switched off at the bar at the top. You can also change the name of this analysis by changing the text in the bar.

#### **Lab:** Calculate Wholesale And Retail Value of our Inventory

**Goal**: Create a new analysis from the Inventory database created earlier that displays a bar graph that sums wholesale value of all prodcuts whose value is _over 8_. The graph data should be sorted in ascending alphabetical order by the product's name.

We'll go over how to produce the end result in a moment, but I encourage you to try this on your own from beginning to end.
It will go over everything we covered in the previous section, along with some extra work that hasn't be covered yet.
Be sure to start at the QuickSight front page.

![Lab 2 Final Result](assets/lab2/Lab2_Final_Result.png)

#### Lab Solution Tips

Click the __New Analysis__ button in the top left of the page.

Select the `product_table_qs` dataset from the list.

Click the __Create analysis__ button at the prompt

Under **Visual Types**, select the __Horizontal bar chart__ option.

Now comes the time to select the proper fields. Begin by selecting __ProductName__ from the **Field List** area. QuickSight is smart enough, and will automatically apply that column to the Y-axis. Next, select the `WholeSaleCost` field. QuickSight will add it as the `Value` for the graph.

The graph is almost complete! Be default, the graph will sort in decending order from based on the `Value`.
To change the order, click the drop down menu under `ProductName`.
There are two icons and the column name that determines the sorting order.
Select the ascending button.
It will adjust the data in a weird way, but that will be fixed once you adjust the column.
Click the drop down menu again and under the column, select `ProductName`.

The final piece is limiting the graph to only show prodcuts whose whole sale cost is over 8.
This is done with QuickSight filters.
On the left side of the page, select **Filter** from the menu.
As expected, there are no filters applied at the moment.
Select the `+` symbol next to **Filters** and select the column `WholeSaleCost`.
Instantly, you'll see the filter applied, but nothing changed!
The filter needs to be adjusted, as it requires a value to be filtered on, but is not set yet.
Select the filter we just created.
Above the red error box, enter in the value 8.
The last thing to adjust is the second drop down in the filter.
It currently says __Equals__, but needs to be changed to __Greater than or equal to__.
To save the filter, select the __Apply__ button at the bottom.
Your graph will be updated, and you can now close out of the filter dialog.

Your analysis is complete!

**Bonus**: Try to add a second __Value__ in your visualization. Add the `RetailCost` data so that both are displayed in the bar graph.

## Cleaning Up

1. Click the QuickSight logo in the upper-left corner to return to the QuickSight homepage:

    ![QuickSight Logo](./assets/lab2/quicksight_logo.png)

1. Click the lower-right corner of each of your analysis cards followed by **Delete** and confirm your intention to **Delete** in the pop-up.